# Análise Exploratória de Dados – Preços de Computadores

Este trabalho consiste numa análise exploratória do dataset **“All Computer Prices”** (Kaggle), que reúne informações sobre computadores (desktop e outros), respetivos componentes e preços. O objetivo principal é compreender como diferentes características de hardware (CPU, RAM, armazenamento, GPU, potência da fonte, etc.) se relacionam com o preço final das máquinas.

Ao longo deste notebook serão desenvolvidas as seguintes etapas:

- **Descrição inicial do dataset**: dimensão, tipos de variáveis e breve caracterização das principais colunas.
- **Limpeza e preparação dos dados**: identificação e tratamento de valores em falta, verificação de linhas duplicadas, correção de tipos de dados e remoção de variáveis irrelevantes.
- **Transformação dos dados**: criação e/ou recodificação de variáveis que facilitem a análise (por exemplo, categorias de componentes ou faixas de preço).
- **Análise exploratória univariada e multivariada**: estudo da distribuição das variáveis individuais e das relações entre componentes e preço (correlações, comparações entre grupos, etc.).
- **Dados agrupados e tabelas resumidas**: utilização de `groupby`, `pivot_table` e tabelas cruzadas para analisar médias, totais e contagens por categoria (por exemplo, por tipo de componente ou gama de computador).
- **Visualização de dados**: aplicação de diferentes tipos de gráficos (histogramas, boxplots, gráficos de barras, dispersão, entre outros) para apoiar a interpretação dos resultados.
- **Síntese e conclusões**: resumo dos principais padrões observados no dataset e possíveis interpretações sobre a influência dos componentes no preço dos computadores.

Num segundo notebook será ainda desenvolvido um **dashboard interativo** com recurso à livraria `panel`, permitindo explorar os dados de forma dinâmica (por exemplo, filtrando por características específicas e visualizando a variação de preços).


## 1. Carregamento das bibliotecas e do dataset

Nesta primeira etapa são carregadas as bibliotecas necessárias para a análise e é importado o ficheiro com os dados. Em seguida, é criada uma cópia do dataset original e visualizadas as primeiras linhas da tabela, de forma a confirmar que o ficheiro foi lido corretamente e a ter um primeiro contacto com a estrutura dos dados.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# para ver gráficos dentro do notebook
%matplotlib inline

# carregar o ficheiro (troca o nome pelo teu)
df = pd.read_csv("dados.csv")  # se for .xlsx, usa pd.read_excel("dados.xlsx")

df_original = df.copy()  # cópia do original

#carrega as 5 primeiras linhas por defeito
df.head()


,device_type,brand,model,release_year,os,form_factor,cpu_brand,cpu_model,cpu_tier,cpu_cores,...,resolution,refresh_hz,battery_wh,charger_watts,psu_watts,wifi,bluetooth,weight_kg,warranty_months,price
0,Desktop,Samsung,Samsung Forge XDI,2022,Windows,ATX,Intel,Intel i5-11129,3,12,...,2560x1440,90,0,0,750,Wi-Fi 6,5.1,11.00,36,1383.99
1,Laptop,Samsung,Samsung Pro KM8,2022,Windows,Mainstream,Intel,Intel i7-11114,4,12,...,1920x1080,90,56,120,0,Wi-Fi 6,5.3,2.03,12,2274.99
2,Desktop,Lenovo,Lenovo Strix BIE,2024,macOS,SFF,AMD,AMD Ryzen 5 5168,2,8,...,3440x1440,120,0,0,850,Wi-Fi 6,5.0,7.00,24,1879.99
3,Desktop,Dell,Dell Cube AXR,2024,Windows,ATX,AMD,AMD Ryzen 5 7550,2,6,...,3440x1440,120,0,0,650,Wi-Fi 6,5.2,6.00,36,1331.99
4,Laptop,Gigabyte,Gigabyte Pro IX1,2024,Linux,Gaming,AMD,AMD Ryzen 7 6230,5,16,...,2560x1600,90,80,90,0,Wi-Fi 6,5.2,1.50,12,2681.99


## 1.1. Análise inicial e preparação para a limpeza dos dados

Antes de iniciar o processo de limpeza e transformação dos dados, é importante compreender a estrutura geral do dataset. Nesta secção é feita uma análise inicial das dimensões da tabela, dos tipos de dados, da existência de valores em falta e das variáveis disponíveis. Esta análise exploratória preliminar permite identificar potenciais problemas (como colunas com muitos valores em falta ou tipos de dados inadequados) e preparar de forma mais informada os passos seguintes de limpeza e preparação dos dados.


In [2]:
df.shape      # nº de linhas e colunas

(100000, 33)

O comando `df.shape` devolve uma tupla com o número de linhas e o número de colunas do dataset. Este resultado permite ter uma primeira noção da dimensão do ficheiro (quantos registos existem e quantas variáveis estão disponíveis para análise).


In [3]:
df.info()         # tipos de dados, valores em falta

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 33 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   device_type          100000 non-null  object 
 1   brand                100000 non-null  object 
 2   model                100000 non-null  object 
 3   release_year         100000 non-null  int64  
 4   os                   100000 non-null  object 
 5   form_factor          100000 non-null  object 
 6   cpu_brand            100000 non-null  object 
 7   cpu_model            100000 non-null  object 
 8   cpu_tier             100000 non-null  int64  
 9   cpu_cores            100000 non-null  int64  
 10  cpu_threads          100000 non-null  int64  
 11  cpu_base_ghz         100000 non-null  float64
 12  cpu_boost_ghz        100000 non-null  float64
 13  gpu_brand            100000 non-null  object 
 14  gpu_model            100000 non-null  object 
 15  gpu_tier          

In [ ]:
O comando `df.info()` apresenta um resumo da estrutura do DataFrame, incluindo o número de registos, o nome de cada coluna, o tipo de dados associado (por exemplo, `int64`, `float64`, `object`) e o número de valores não nulos em cada variável. Esta informação é útil para identificar desde cedo a presença de valores em falta e possíveis problemas de tipagem (por exemplo, números guardados como texto), que terão de ser tratados na fase de limpeza.


In [4]:
df.describe()     # estatísticas das colunas numéricas

,release_year,cpu_tier,cpu_cores,cpu_threads,cpu_base_ghz,cpu_boost_ghz,gpu_tier,vram_gb,ram_gb,storage_gb,storage_drive_count,display_size_in,refresh_hz,battery_wh,charger_watts,psu_watts,bluetooth,weight_kg,warranty_months,price
count,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.00000,100000.000000
mean,2022.320850,3.153490,10.515740,19.372700,2.591322,3.531310,2.991350,6.152180,39.706400,903.936000,1.524980,20.126655,98.464860,41.813470,61.383450,272.520500,5.084764,4.289699,22.20036,1928.764220
std,2.025761,1.373175,5.044092,9.718426,0.336435,0.350024,1.459643,3.964926,31.902684,774.243654,0.797284,6.709577,43.301652,35.868841,62.795034,354.686355,0.245977,3.814628,10.23190,580.492689
min,2018.000000,1.000000,4.000000,4.000000,2.000000,2.800000,1.000000,0.000000,8.000000,256.000000,1.000000,13.300000,60.000000,0.000000,0.000000,0.000000,4.200000,0.920000,12.00000,372.990000
25%,2021.000000,2.000000,6.000000,12.000000,2.400000,3.300000,2.000000,4.000000,16.000000,512.000000,1.000000,14.000000,60.000000,0.000000,0.000000,0.000000,5.000000,1.500000,12.00000,1503.990000
50%,2023.000000,3.000000,8.000000,16.000000,2.600000,3.500000,3.000000,6.000000,32.000000,512.000000,1.000000,16.000000,90.000000,56.000000,65.000000,0.000000,5.100000,2.000000,24.00000,1863.990000
75%,2024.000000,4.000000,14.000000,24.000000,2.800000,3.800000,4.000000,8.000000,64.000000,1024.000000,2.000000,27.000000,120.000000,70.000000,90.000000,650.000000,5.200000,7.000000,24.00000,2287.990000
max,2025.000000,6.000000,28.000000,56.000000,3.400000,4.500000,6.000000,16.000000,144.000000,4096.000000,4.000000,34.000000,240.000000,99.000000,240.000000,1200.000000,5.300000,16.000000,48.00000,10984.990000


O comando `df.describe()` calcula estatísticas descritivas básicas para as variáveis numéricas do dataset, como a média, desvio padrão, valores mínimo e máximo, e os quartis (25%, 50% e 75%). Estas estatísticas permitem ter uma visão geral da distribuição dos dados numéricos, identificar ordens de grandeza e possíveis valores extremos (outliers) que poderão influenciar as análises posteriores.


In [5]:

df.columns        # nomes das colunas


Index(['device_type', 'brand', 'model', 'release_year', 'os', 'form_factor',
       'cpu_brand', 'cpu_model', 'cpu_tier', 'cpu_cores', 'cpu_threads',
       'cpu_base_ghz', 'cpu_boost_ghz', 'gpu_brand', 'gpu_model', 'gpu_tier',
       'vram_gb', 'ram_gb', 'storage_type', 'storage_gb',
       'storage_drive_count', 'display_type', 'display_size_in', 'resolution',
       'refresh_hz', 'battery_wh', 'charger_watts', 'psu_watts', 'wifi',
       'bluetooth', 'weight_kg', 'warranty_months', 'price'],
      dtype='object')

O comando `df.columns` devolve a lista com os nomes de todas as colunas do DataFrame. A visualização destes nomes ajuda a perceber que tipo de informação está disponível (por exemplo, especificações de hardware, preço, características adicionais) e a decidir quais as variáveis que serão mais relevantes para a análise exploratória.


## 2. Limpeza de dados

Depois de compreender a estrutura geral do dataset, é necessário verificar a qualidade dos dados disponíveis. Nesta fase é feita a identificação de valores em falta e de eventuais registos duplicados. Estes problemas podem afetar as análises estatísticas e as conclusões, pelo que é importante detetá-los e, quando necessário, aplicar estratégias de correção (remoção de linhas, substituição de valores, etc.).


In [9]:
#valores em falta por coluna em ordem descendente
df.isna().sum().sort_values(ascending=False)


device_type            0
brand                  0
model                  0
release_year           0
os                     0
form_factor            0
cpu_brand              0
cpu_model              0
cpu_tier               0
cpu_cores              0
cpu_threads            0
cpu_base_ghz           0
cpu_boost_ghz          0
gpu_brand              0
gpu_model              0
gpu_tier               0
vram_gb                0
ram_gb                 0
storage_type           0
storage_gb             0
storage_drive_count    0
display_type           0
display_size_in        0
resolution             0
refresh_hz             0
battery_wh             0
charger_watts          0
psu_watts              0
wifi                   0
bluetooth              0
weight_kg              0
warranty_months        0
price                  0
dtype: int64

O comando `df.isna().sum().sort_values(ascending=False)` calcula, para cada coluna, o número de valores em falta (`NaN`) e apresenta o resultado ordenado por ordem decrescente. Desta forma, é possível identificar rapidamente quais as variáveis mais afetadas por valores em falta e que poderão necessitar de tratamento específico (por exemplo, remoção de registos, substituição por média/mediana ou criação de categorias como "desconhecido").


In [7]:
# linhas duplicadas
df.duplicated().sum()

#Caso ouvesse repetidos ---> df = df.drop_duplicates()


np.int64(0)

O comando `df.duplicated().sum()` devolve o número de linhas duplicadas no dataset, ou seja, registos que são exatamente iguais noutros pontos da tabela. A existência de duplicados pode distorcer as análises (por exemplo, ao contar ou calcular médias), pelo que, caso fossem detetadas linhas repetidas, estas poderiam ser removidas com o comando `df = df.drop_duplicates()`. No presente caso, o resultado obtido indica se é ou não necessário aplicar essa remoção.

No entanto, verificou-se que o número de linhas duplicadas é igual a 0, pelo que não foi necessário remover qualquer registo.

In [15]:
num = df.select_dtypes(include=np.number)       # só colunas numéricas
zeros_por_coluna = num.eq(0).sum().sort_values(ascending=False)
zeros_por_coluna

psu_watts              59844
battery_wh             40156
charger_watts          40156
vram_gb                13389
cpu_threads                0
cpu_cores                  0
cpu_tier                   0
release_year               0
ram_gb                     0
cpu_base_ghz               0
cpu_boost_ghz              0
gpu_tier                   0
display_size_in            0
storage_drive_count        0
storage_gb                 0
refresh_hz                 0
bluetooth                  0
weight_kg                  0
warranty_months            0
price                      0
dtype: int64


Foi analisada a frequência de valores iguais a 0 nas colunas numéricas do dataset.  
Este passo permite identificar variáveis onde o valor 0 pode significar ausência de componente, valor desconhecido ou um caso especial (por exemplo, computadores sem bateria).  

As colunas com maior número de zeros serão analisadas com mais detalhe, para decidir se estes valores são válidos ou se devem ser tratados na fase de limpeza dos dados.


In [13]:
df["battery_wh"].value_counts().head()

battery_wh
0     40156
70    13110
60    11983
80    10659
56     9135
Name: count, dtype: int64

In [14]:
df["psu_watts"].value_counts().head()

psu_watts
0      59844
650     8831
750     7923
550     7179
850     5625
Name: count, dtype: int64

In [23]:
df.loc[df["battery_wh"] == 0].head()


,device_type,brand,model,release_year,os,form_factor,cpu_brand,cpu_model,cpu_tier,cpu_cores,...,resolution,refresh_hz,battery_wh,charger_watts,psu_watts,wifi,bluetooth,weight_kg,warranty_months,price
0,Desktop,Samsung,Samsung Forge XDI,2022,Windows,ATX,Intel,Intel i5-11129,3,12,...,2560x1440,90,0,0,750,Wi-Fi 6,5.1,11.0,36,1383.99
2,Desktop,Lenovo,Lenovo Strix BIE,2024,macOS,SFF,AMD,AMD Ryzen 5 5168,2,8,...,3440x1440,120,0,0,850,Wi-Fi 6,5.0,7.0,24,1879.99
3,Desktop,Dell,Dell Cube AXR,2024,Windows,ATX,AMD,AMD Ryzen 5 7550,2,6,...,3440x1440,120,0,0,650,Wi-Fi 6,5.2,6.0,36,1331.99
5,Desktop,MSI,MSI Think KSG,2025,Windows,ATX,Intel,Intel i7-10369,5,16,...,2560x1440,90,0,0,1000,Wi-Fi 5,5.0,9.0,36,2751.99
6,Desktop,Apple,Apple Arena R5Q,2024,Windows,ATX,Apple,Apple M2,2,6,...,2560x1440,60,0,0,850,Wi-Fi 6,5.1,9.0,24,1609.99


In [25]:
df.loc[df["battery_wh"] == 0, "device_type"].value_counts(normalize=True)


device_type
Desktop    1.0
Name: proportion, dtype: float64

In [22]:
df.loc[df["psu_watts"] == 0].head()


,device_type,brand,model,release_year,os,form_factor,cpu_brand,cpu_model,cpu_tier,cpu_cores,...,resolution,refresh_hz,battery_wh,charger_watts,psu_watts,wifi,bluetooth,weight_kg,warranty_months,price
1,Laptop,Samsung,Samsung Pro KM8,2022,Windows,Mainstream,Intel,Intel i7-11114,4,12,...,1920x1080,90,56,120,0,Wi-Fi 6,5.3,2.03,12,2274.99
4,Laptop,Gigabyte,Gigabyte Pro IX1,2024,Linux,Gaming,AMD,AMD Ryzen 7 6230,5,16,...,2560x1600,90,80,90,0,Wi-Fi 6,5.2,1.50,12,2681.99
8,Laptop,Dell,Dell Creator GIQ,2024,Windows,Mainstream,Intel,Intel i9-14473,6,26,...,2560x1600,60,80,240,0,Wi-Fi 5,5.0,1.17,48,2953.99
9,Laptop,Lenovo,Lenovo Blade MIZ,2025,Windows,Ultrabook,AMD,AMD Ryzen 3 4374,1,4,...,3840x2160,120,60,45,0,Wi-Fi 6,5.3,1.50,24,1653.99
10,Laptop,HP,HP Pro 6XS,2023,Windows,Mainstream,Intel,Intel i5-13209,2,6,...,1920x1080,165,70,90,0,Wi-Fi 6E,5.2,1.24,36,1371.99


In [ ]:
# entre as linhas onde psu_watts == 0,
# quantos são desktop / laptop?
df.loc[df["psu_watts"] == 0, "device_type"].value_counts(normalize=True)


device_type
Laptop    59844
Name: count, dtype: int64

## 3. Transformação de Dados